# Exploration 4. 작사가 인공지능 만들기

본 과제는 파이썬의 시퀀스 자료형과 순환신경망(RNN)을 이용하여 인공지능이 문장을 읽어들이는 방식을 이해하고, 이 인공지능이 작문을 하도록 돕는다.

## 과제 평가 기준
1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?<br>
   텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?<br>
2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?<br>
   특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가?<br>
3. 텍스트 생성모델이 안정적으로 학습되었는가?<br>
   텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?<br>
   
* * *

## 순환신경망(RNN; Recurrent Neural Network)에 대하여
* 시계열 데이터와 같이 시간의 흐름에 따라 변화하는 데이터를 학습하기 위한 인공신경망으로, 과거의 출력 데이터를 재귀(자기 자신을 참조한다는 의미로, 현재와 이전의 결과가 연관성을 가짐을 의미)적으로 참조
* 주로 앞뒤 데이터 간의 연관성이 있는 데이터셋에 사용되는 모델로, 음성 인식 및 번역 시스템에 많이 사용되고 있음
* 참고 링크 : [인공지능의 이해 (5/6): 순환 신경망(RNN)](https://brunch.co.kr/@linecard/324)

## 데이터를 읽어온 후 정제하기
glob 를 활용하여 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장한다. 그 다음 저장된 문장들을 preprocess_sentence() 함수를 활용하여 데이터를 정제한다. 문장을 토큰화 했을 시 토큰의 개수가 15개를 넘는 문장을 학습에서 제외한다.

In [13]:
import glob
import os, re 
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)
raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담는다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:9]) # 앞에서부터 10라인만 화면에 출력

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?", 'It goes like this', 'The fourth, the fifth', 'The minor fall, the major lift', 'The baffled king composing Hallelujah Hallelujah', 'Hallelujah', 'Hallelujah']


In [14]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장 건너뛰기
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장 건너뛰기

    if idx > 9: break   # 먼저 문장 10개만 확인하도록 하자.
        
    print(sentence)

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof


In [15]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백 삭제
#     2. 특수문자 양쪽에 공백 삽입
#     3. 여러개의 공백은 하나의 공백으로 변환
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 변환
#     5. 다시 양쪽 공백을 삭제
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 문장 필터링 결과 확인
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [16]:
# 정제된 문장 수집
corpus = []

for sentence in raw_corpus:
    # 원하지 않는 문장 건너뛰기
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제된 문장 담기
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 10개의 정제된 결과 확인
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

## 훈련/평가 데이터셋 분리
tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리한다. 단어장의 크기는 12,000 단어 이상이며, 총 데이터의 20%를 평가 데이터셋으로 설정한다.

In [17]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용
def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer 생성
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 변환
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words = 12000, 
        filters = ' ',
        oov_token = "<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춤
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줌
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding = 'post', maxlen = 15)
    # 토큰의 개수가 15개를 초과하는 문장 제외
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 2639 ...    0    0    0]
 [   2   36    7 ...   43    3    0]
 ...
 [   5   22    9 ...   10 1013    3]
 [  37   15 9049 ...  877  647    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f3ce5ce62b0>


In [18]:
print(tensor[:3, :10])

[[   2   50    5   91  297   65   57    9  969 6042]
 [   2   17 2639  873    4    8   11 6043    6  329]
 [   2   36    7   37   15  164  282   28  299    4]]


In [19]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [21]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높음
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

# 평가 데이터셋 분리

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size = 0.2, 
                                                          shuffle = True, 
                                                          random_state = 32)

print("Source Train : ", enc_train.shape)
print("Target Train : ", dec_train.shape)

[   2   50    5   91  297   65   57    9  969 6042    3    0    0    0]
[  50    5   91  297   65   57    9  969 6042    3    0    0    0    0]
Source Train :  (140599, 14)
Target Train :  (140599, 14)


In [22]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 생성
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder = True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

## 인공지능 모델 학습시키기
tf.keras.Model을 Subclassing하는 방식으로 모델을 만든 후, 모델의 Embedding Size와 Hidden Size를 조절해 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계한다. 
<img src="https://d3s0tskafalll9.cloudfront.net/media/images/E-12-4.max-800x600.png"><br>

## LSTM(Long Short-Term Memory, LSTM)
* 전통적인 RNN(이를 바닐라 RNN이라고 한다)의 경우, 비교적 짧은 시퀀스(Sequence)에 대해서만 효과를 보이는 문제점이 존재
* 기존 RNN의 시점(time step)이 길어질수록 앞의 정보가 뒤로 충분히 전달되지 못한다는 문제가 있음
* LSTM은 기존 RNN이 가진 이러한 장기 의존성 문제(the problem of Long-Term Dependencies)를 보완
* 은닉층의 메모리 셀에 입력, 망각, 출력 게이트를 추가하여 불필요한 기억을 지우고, 기억해야 할 것들을 결정함
* 참고 링크 : [2) 장단기 메모리(Long Short-Term Memory, LSTM)](https://wikidocs.net/22888)

In [23]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 1024
lyricist = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [24]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 삽입
lyricist(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-6.63743485e-05,  3.62666266e-04, -9.97726511e-06, ...,
         -1.35906244e-04, -1.20688135e-04, -2.34706022e-04],
        [-2.55395571e-04,  7.42503325e-04,  1.43708079e-04, ...,
         -4.24297265e-04, -3.50188056e-04, -1.67167746e-04],
        [-5.02977986e-04,  9.70414199e-04,  3.81613558e-04, ...,
         -5.39670582e-04, -3.15920683e-04, -5.85545349e-05],
        ...,
        [ 1.12316309e-04, -3.75681790e-04, -1.11192814e-03, ...,
         -5.65824623e-04,  6.93646085e-04,  1.68304436e-03],
        [-1.42699661e-04, -7.29323016e-04, -9.44326282e-04, ...,
         -6.32083102e-04,  1.08422968e-03,  1.67148735e-03],
        [-3.72330978e-04, -1.04658341e-03, -7.34705245e-04, ...,
         -7.67060381e-04,  1.53901847e-03,  1.68981694e-03]],

       [[-6.63743485e-05,  3.62666266e-04, -9.97726511e-06, ...,
         -1.35906244e-04, -1.20688135e-04, -2.34706022e-04],
        [-2.40007852e-04,  6.01882988e-04, -2

In [25]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits = True,
    reduction = 'none'
)

lyricist.compile(loss=loss, optimizer=optimizer)
lyricist.fit(dataset, epochs=10)

Epoch 1/10
686/686 [==============================] - 123s 173ms/step - loss: 3.5766
Epoch 2/10
686/686 [==============================] - 126s 183ms/step - loss: 3.0824
Epoch 3/10
686/686 [==============================] - 126s 183ms/step - loss: 2.8738
Epoch 4/10
686/686 [==============================] - 126s 183ms/step - loss: 2.7073
Epoch 5/10
686/686 [==============================] - 126s 184ms/step - loss: 2.5618
Epoch 6/10
686/686 [==============================] - 126s 184ms/step - loss: 2.4298
Epoch 7/10
686/686 [==============================] - 127s 184ms/step - loss: 2.3052
Epoch 8/10
686/686 [==============================] - 127s 185ms/step - loss: 2.1875
Epoch 9/10
686/686 [==============================] - 127s 185ms/step - loss: 2.0743
Epoch 10/10
686/686 [==============================] - 127s 185ms/step - loss: 1.9662


val_loss의 값이 1.9662로 감소한 것을 확인할 수 있다.

## 가사 생성하기

generate_text 함수를 이용하여 모델에게 시작 문장을 전달하고, 그 문장을 바탕으로 모델이 작사를 하도록 한다. 

In [30]:
def generate_text(lyricist, tokenizer, init_sentence="<start>", max_len=15):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 생성
    #    1. 입력받은 문장의 텐서를 입력
    #    2. 예측된 값 중 가장 높은 확률인 word index를 검출
    #    3. 2에서 예측된 word index를 문장 뒤에 이어 붙임
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성 종료
    while True:
        # 1
        predict = lyricist(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [31]:
generate_text(lyricist, tokenizer, init_sentence="<start> You are", max_len=15)

'<start> you are the one <end> '

In [32]:
generate_text(lyricist, tokenizer, init_sentence="<start> I love", max_len=15)

'<start> i love you <end> '

In [33]:
generate_text(lyricist, tokenizer, init_sentence="<start> I was in", max_len=15)

'<start> i was in the dominican big papi ortiz <end> '

## 회고
* 순환신경망 모델의 구조를 좀 더 면밀히 학습해야 할 필요성을 느꼈다. 다행히도 제시한 과제에 맞게 결과물이 도출되었지만 순환신경망 모델의 구조에 대한 이해가 아직 제대로 정립이 되지 않아 복잡하게 느껴져 과제 진행에 있어 설명하는 것에 한계를 체감하였다.